# Top End Assembly Vibration Study

As described in [OBS-467], we have been noticing strong vibrations on the Top End Assembly (TEA) of the Simonyi Survey Telescope (SST). 
The first time we saw these vibrations was on March 9th (Saturday) and on March 10th (Sunday). 

Accordingly to [SITCOM-1285], on March 15th (Friday), we ran soak tests using the Rotator at different configurations. 
We want to investigate any possible vibrations during that period to confirm the hypothesis that the Rotator could be causing this vibration.

In addition to both cases, on Marth 21th (Thursday), we executed [BLOCK-197] a couple of times. 
[OBS-498] reports multiple faults on the hexapods associated with the compensation mode. 
It also indicates that we might have vibrations during its execution, we need to review it. 

Finally, on March 25th, we executed [BLOCK-197] twice. 
First with the rotator in ENABLED. 
Second with the rotator in STANDBY and with its cabinet turned off. 
We want to compare both cases. 

[BLOCK-197]: https://rubinobs.atlassian.net/browse/BLOCK-197
[OBS-467]: https://rubinobs.atlassian.net/browse/OBS-467
[OBS-498]: https://rubinobs.atlassian.net/browse/OBS-498
[SITCOM-1285]: https://rubinobs.atlassian.net/browse/SITCOM-1285

## Notebook Preparation

In [ ]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [ ]:
import asyncio
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sys
import time
import warnings

from argparse import Namespace
from astropy.time import Time, TimeDelta
from scipy.fft import fft, fftfreq
from scipy.signal import detrend, get_window, welch
from lsst.summit.utils.blockUtils import BlockParser
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData
from lsst_efd_client.efd_helper import merge_packed_time_series


# Ignore the many warning messages from ``merge_packed_time_series``
warnings.simplefilter(action="ignore", category=FutureWarning)

# Create an EFD client
client = makeEfdClient()

# Global variables definition
sensorNames = [
    "SST top end ring +x -y",
    "SST top end ring -x -y",
    "SST spider spindle",
    "SST M2 surrogate",
]
sensorNamesShort = ["ter_pxmy", "ter_mxmy", "spider", "m2surr"]

# Create a folder for plots
os.makedirs("./plots", exist_ok=True)

## OBS-467 - The initial problem

In [ ]:
start = Time("2024-03-09T10:00:00Z", scale="utc", format="isot")
end = Time("2024-03-11T06:00:00Z", scale="utc", format="isot")

df_rot = getEfdData(
    client,
    "lsst.sal.MTRotator.rotation",
    columns=["actualPosition"],
    begin=start,
    end=end,
)

df_m2 = getEfdData(
    client,
    "lsst.sal.MTM2.positionIMS",
    columns=["x", "y", "z"],
    begin=start,
    end=end,
)

df_hex = getEfdData(
    client,
    "lsst.sal.MTHexapod.application",
    columns=[f"position{i}" for i in range(3)] + ["salIndex"],
    begin=start,
    end=end,
)

In [ ]:
df_m2 = df_m2 - df_m2.rolling("1s").mean()
df_camhex = df_hex[df_hex.salIndex == 1]
df_m2hex = df_hex[df_hex.salIndex == 2]

<div class="alert alert-warning">
The plot below takes a 2.5 min to be executed. 
</div>

In [ ]:
%matplotlib inline
fig, (ax_m2, ax_camhex, ax_m2hex, ax_rot) = plt.subplots(
    nrows=4, figsize=(16, 7), sharex=True
)

ax_m2.plot(df_m2.x, label="x")
ax_m2.plot(df_m2.y, label="y")
ax_m2.plot(df_m2.z, label="z")
ax_m2.grid(":", alpha=0.25)
ax_m2.set_ylabel("M2 IMS\n Position [um]")
ax_m2.legend(loc="upper left")

ax_camhex.plot(df_camhex.position0, label="x")
ax_camhex.plot(df_camhex.position1, label="y")
ax_camhex.plot(df_camhex.position2, label="z")
ax_camhex.grid(":", alpha=0.25)
ax_camhex.set_ylabel("CamHex\n Position [um]")
ax_camhex.legend(loc="upper left")

ax_m2hex.plot(df_m2hex.position0, label="x")
ax_m2hex.plot(df_m2hex.position1, label="y")
ax_m2hex.plot(df_m2hex.position2, label="z")
ax_m2hex.grid(":", alpha=0.25)
ax_m2hex.set_ylabel("M2\n Position [um]")
ax_m2hex.legend(loc="upper left")

ax_rot.plot(df_rot.actualPosition * 1e3, label="Actual Position")
ax_rot.grid(":", alpha=0.25)
ax_rot.set_ylabel("Rotator Angle\n [$10^-3$ deg]")
ax_rot.set_xlim(df_rot.index[0], df_rot.index[-1])
ax_rot.set_xlabel("Time [utc]")
ax_rot.legend(loc="upper left")

fig.suptitle(
    "OBS-467 Investigation - M2/Rotator/Hexapods Vibrrations\n"
    f"From {start.to_value('iso', subfmt='date_hm')} to {end.to_value('iso', subfmt='date_hm')}"
)

fig.autofmt_xdate()
fig.tight_layout()

fig.savefig("./plots/obs467_investigation.png")
plt.show()

The plots above show a low amplitude vibration on the Rotator and on M2 from 00:40 to 15:00 UTC on March 10th.  
Following the logs in OLE, the first problematic measurements using the Laser Tracker started to appear around 00:44. 
This is consistent with what we see above. 

We checked the vibrations on the rotator actual position on Chronograf, we found that the exact time that it started was at `00:37:55`.
At `14:41:13`, the amplitude of these oscillations are smaller and last until `16:08:41`.  

Alysha confirmed that all the vibrations/oscillations after 17h UTC come from other tests and are unrelated to the sounds we hear when going to Level 8. 

Another important aspect to consider are the amplitudes of the vibrations. 
A quick eye inspection show that the oscillation on the rotator have an amplitude of about 0.1 x 10^-3 degrees. 
The M2 have oscillations around 1 um. 
None of these two justify the 2 mm displacement that we were measuring with the Laser Tracker.

When looking at the hexapods data, we can see the warm-up sequence twice. 
First, at around 18h UTC on March 9th and, second, near 14h UTC on March 10th. 
These two data sets give us an example of the range of movement on the hexapods. 

We can also see both hexapods being exercised quite a lot near the time when the vibrations/oscillations started. 

Several questions remain for this time window and we can use the following events/commands to investigate a bit more:
- summary states for the hexapods and the rotator
- drive states for the hexapods and the rotator
- when was the compensation mode enabled/disabled?

## BLOCK-197 w/ Rotator PXI powered On

[BLOCK-197] is the test we were running when the vibration happened. 
We want to compare its execution with the Rotator PXI and drives powered on versus powered off. 
We executed both tests on the night of March 29th, 2024. 
The day obs is `20240329`. The code below grabs the scripts ran for this block. 

[BLOCK-197]: https://rubinobs.atlassian.net/browse/BLOCK-197

In [ ]:
day_obs = 20240329
block_id = 197

block_parser = BlockParser(day_obs)
blocks = block_parser.getBlockNums()
list_of_blocks = block_parser.getSeqNums(block_id)

In [ ]:
for num in list_of_blocks:
    block = block_parser.getBlockInfo(block_id, num)
    print(f"{num:02} {block.begin.iso} {block.end.iso}")

Considering the timestamps and the number of scripts, it is reasonable to say that `seqNum` from 1 to 10 belong to BLOCK-197's first execution (rotator ON).  
The rest belong to the second execution (rotator OFF). 

In [ ]:
def query_efd_data(start, end):
    dfs = Namespace()

    dfs.start = start
    dfs.end = end

    dfs.rot = getEfdData(
        client,
        "lsst.sal.MTRotator.rotation",
        columns=["actualPosition"],
        begin=start,
        end=end,
    )

    dfs.m2 = getEfdData(
        client,
        "lsst.sal.MTM2.positionIMS",
        columns=["x", "y", "z"],
        begin=start,
        end=end,
    )
    # dfs.m2 = dfs.m2 - dfs.m2.mean()
    dfs.m2["times"] = dfs.m2.index

    dfs.hexs = getEfdData(
        client,
        "lsst.sal.MTHexapod.application",
        columns=[f"position{i}" for i in range(3)] + ["salIndex"],
        begin=start,
        end=end,
    )
    dfs.camhex = dfs.hexs[dfs.hexs.salIndex == 1]
    dfs.m2hex = dfs.hexs[dfs.hexs.salIndex == 2]

    packed_ess_df = getEfdData(
        client,
        "lsst.sal.ESS.accelerometer",
        columns="*",
        begin=start,
        end=end,
    )

    for short_name, long_name in zip(sensorNamesShort, sensorNames):
        fields_df = {}
        sub_df = packed_ess_df.loc[packed_ess_df.sensorName == long_name]

        baseFields = ["accelerationX", "accelerationY", "accelerationZ"]

        for base_field in baseFields:
            fields_df[base_field] = merge_packed_time_series(
                sub_df,
                base_field,
                stride=1,
                ref_timestamp_col="timestamp",
                fmt="unix_tai",
                scale="tai",
            )
            fields_df[base_field] = fields_df[base_field].sort_index()

        sub_df = pd.merge_asof(
            pd.merge_asof(
                fields_df[baseFields[0]],
                fields_df[baseFields[1]],
                left_index=True,
                right_index=True,
            ),
            fields_df[baseFields[2]],
            left_index=True,
            right_index=True,
        )
        sub_df = sub_df - sub_df.mean()
        setattr(dfs, short_name, sub_df)

    return dfs

In [ ]:
def plot_dataframes(dfs, title, filename):
    fig, (
        ax_m2,
        ax_camhex,
        ax_m2hex,
        ax_rot,
        ax_spider,
        ax_m2surr,
        ax_ter_pxmy,
        ax_ter_mxmy,
    ) = plt.subplots(nrows=8, figsize=(16, 16), sharex=True, num=filename)

    ax_m2.plot(dfs.m2.x, label="x")
    ax_m2.plot(dfs.m2.y, label="y")
    ax_m2.plot(dfs.m2.z, label="z")
    ax_m2.grid(":", alpha=0.25)
    ax_m2.set_xlim(dfs.start.to_datetime(), dfs.end.to_datetime())
    ax_m2.set_ylabel("M2 IMS\n Position [um]")
    ax_m2.legend(loc="upper left")

    ax_camhex.plot(dfs.camhex.position0, label="x")
    ax_camhex.plot(dfs.camhex.position1, label="y")
    ax_camhex.plot(dfs.camhex.position2, label="z")
    ax_camhex.grid(":", alpha=0.25)
    ax_camhex.set_ylabel("CamHex\n Position [um]")
    ax_camhex.legend(loc="upper left")

    ax_m2hex.plot(dfs.m2hex.position0, label="x")
    ax_m2hex.plot(dfs.m2hex.position1, label="y")
    ax_m2hex.plot(dfs.m2hex.position2, label="z")
    ax_m2hex.grid(":", alpha=0.25)
    ax_m2hex.set_ylabel("M2\n Position [um]")
    ax_m2hex.legend(loc="upper left")

    try:
        ax_rot.plot(dfs.rot.actualPosition * 1e3, label="Actual Position")
    except (AttributeError, IndexError):
        pass

    ax_rot.grid(":", alpha=0.25)
    ax_rot.set_ylabel("Rotator Angle\n [$10^-3$ deg]")
    ax_rot.set_xlabel("Time [utc]")
    ax_rot.legend(loc="upper left")

    ax_spider.plot(dfs.spider.accelerationX, label="x", alpha=0.3)
    ax_spider.plot(dfs.spider.accelerationY, label="y", alpha=0.3)
    ax_spider.plot(dfs.spider.accelerationZ, label="z", alpha=0.3)
    ax_spider.grid(":", alpha=0.25)
    ax_spider.set_ylabel("SST Spider\nAcceleration [$m.s^{-2}$]")
    ax_spider.legend(loc="upper left")

    ax_m2surr.plot(dfs.m2surr.accelerationX, label="x", alpha=0.3)
    ax_m2surr.plot(dfs.m2surr.accelerationY, label="y", alpha=0.3)
    ax_m2surr.plot(dfs.m2surr.accelerationZ, label="z", alpha=0.3)
    ax_m2surr.grid(":", alpha=0.25)
    ax_m2surr.set_ylabel("SST M2 Surrogate\nAcceleration [$m.s^{-2}$]")
    ax_m2surr.legend(loc="upper left")

    ax_ter_mxmy.plot(dfs.ter_mxmy.accelerationX, label="x", alpha=0.3)
    ax_ter_mxmy.plot(dfs.ter_mxmy.accelerationY, label="y", alpha=0.3)
    ax_ter_mxmy.plot(dfs.ter_mxmy.accelerationZ, label="z", alpha=0.3)
    ax_ter_mxmy.grid(":", alpha=0.25)
    ax_ter_mxmy.set_ylabel("SST top end ring -x -y\nAcceleration [$m.s^{-2}$]")
    ax_ter_mxmy.legend(loc="upper left")

    ax_ter_pxmy.plot(dfs.ter_pxmy.accelerationX, label="x", alpha=0.3)
    ax_ter_pxmy.plot(dfs.ter_pxmy.accelerationY, label="y", alpha=0.3)
    ax_ter_pxmy.plot(dfs.ter_pxmy.accelerationZ, label="z", alpha=0.3)
    ax_ter_pxmy.grid(":", alpha=0.25)
    ax_ter_pxmy.set_ylabel("SST top end ring +x -y\nAcceleration [$m.s^{-2}$]")
    ax_ter_pxmy.legend(loc="upper left")

    fig.suptitle(
        f"{title}\n"
        f"From {start.to_value('iso', subfmt='date_hm')} to {end.to_value('iso', subfmt='date_hm')}"
    )

    fig.autofmt_xdate()
    fig.tight_layout()

    fig.savefig(f"./plots/{filename}")
    plt.show()

In [ ]:
start = block_parser.getBlockInfo(block_id, 1).begin
end = block_parser.getBlockInfo(block_id, 10).end

dfs_on = query_efd_data(start, end)

In [ ]:
%matplotlib inline
plot_dataframes(
    dfs_on,
    title="OBS-467 Investigation - Running BLOCK-197 w/ Rotator On",
    filename="block197_rotator_on.png",
)

## BLOCK-197 w/ Rotator PXI powered Off


In [ ]:
start = block_parser.getBlockInfo(block_id, 11).begin
end = block_parser.getBlockInfo(block_id, 20).end

dfs_off = query_efd_data(start, end)

In [ ]:
%matplotlib inline
plot_dataframes(
    dfs_off,
    title="OBS-467 Investigation - Running BLOCK-197 w/ Rotator OFF",
    filename="block197_rotator_off.png",
)

## BLOCK-197 - Rotator On/Off comparison

The plots above shows how the Rotator and the Accelerometers respond to the hexapod movements. 
We are interested in knowing the amplitude of the vibrations and their main frequencies. 

Assuming, for simplicity, that the vibration was tangencial to the circle containing the SMRs.
For the Camera SMRs, a displacement of 2 mm in a circle of 850 mm would need the rotator to oscillate with an amplitude of 0.135 deg.
For the M2 SMRs, a displacement of 2 mm in a circle of 1740 mm would need the rotator to oscillate with an amplitude of 0.065 deg.

We could lock on M2 SMRs. Oscillations about 0.1 mm. In contrast of Cam SMRs, of 2 mm. 

In [ ]:
def calculate_amplitudes(dfs, sensorNamesShort, sensorNames):
    # Initialize an empty list to hold the data
    data = []

    # Iterate over the sensor names and dataframes
    for short_name, long_name in zip(sensorNamesShort, sensorNames):
        df = getattr(dfs, short_name)

        # Calculate the amplitude and RMS for each direction
        data.append(
            {
                "Accelerometer": long_name,
                "Direction": "X",
                "Amplitude": np.ptp(df.accelerationX),
                "RMS": np.sqrt(np.mean(df.accelerationX**2)),
            }
        )
        data.append(
            {
                "Accelerometer": long_name,
                "Direction": "Y",
                "Amplitude": np.ptp(df.accelerationY),
                "RMS": np.sqrt(np.mean(df.accelerationY**2)),
            }
        )
        data.append(
            {
                "Accelerometer": long_name,
                "Direction": "Z",
                "Amplitude": np.ptp(df.accelerationZ),
                "RMS": np.sqrt(np.mean(df.accelerationZ**2)),
            }
        )

    # Convert the list of data into a DataFrame
    results_df = pd.DataFrame(data)

    # Optionally, you might want to set a multi-level index for easier querying
    # results_df.set_index(['Accelerometer', 'Direction'], inplace=True)

    return results_df

In [ ]:
pd.set_option("display.width", 1000)

results_df_off = calculate_amplitudes(dfs_off, sensorNamesShort, sensorNames)
results_df_on = calculate_amplitudes(dfs_on, sensorNamesShort, sensorNames)

results_df_off["State"] = "Off"
results_df_on["State"] = "On"

# Merge the two DataFrames
merged_df = pd.concat([results_df_off, results_df_on])

# Pivot the DataFrame to better compare 'On' vs 'Off' states for each accelerometer and direction
pivot_table = merged_df.pivot_table(
    index=["Accelerometer", "Direction"], columns="State", values=["Amplitude", "RMS"]
)

# Reset index if you prefer a flat structure
pivot_table.reset_index(inplace=True)

# Display the pivot table
print(pivot_table)

## Frequency Analysis

In [ ]:
def frequency_analysis_side_by_side(data_off, data_on, long_name, columns, filename):
    start = min(data_off.index[0], data_on.index[0])
    end = max(data_off.index[-1], data_on.index[-1])

    fig, axs = plt.subplots(1, 2, figsize=(20, 6), sharey=True)
    fig.suptitle(
        f"FFT of {long_name} - Off vs. On\n"
        f"From {start.strftime('%Y-%m-%d %H:%M:%S')} to {end.strftime('%Y-%m-%d %H:%M:%S')}"
    )

    # Define a function to perform FFT and plot for a given dataset
    def plot_fft(data, ax, title):
        data["relative_time"] = data["times"] - data["times"].min()

        for i, col in enumerate(columns):
            detrended_acc = detrend(data[col])
            window = get_window("hamming", len(detrended_acc))
            windowed_acc = detrended_acc * window

            fft_acc = fft(windowed_acc)
            fft_freqs = fftfreq(
                len(windowed_acc),
                (data["relative_time"].iloc[1] - data["relative_time"].iloc[0]),
            )

            positive_freqs = fft_freqs[: len(fft_freqs) // 2]
            positive_fft = np.abs(fft_acc[: len(fft_acc) // 2])

            ax.plot(positive_freqs, positive_fft + 500 * i, label=col, alpha=0.5)

        ax.set_title(title)
        ax.set_xlabel("Frequency (Hz)")
        ax.set_ylabel("Magnitude")
        ax.grid(True)
        ax.legend()

    # Plot for 'Off' state
    plot_fft(data_off, axs[0], "Off")
    # Plot for 'On' state
    plot_fft(data_on, axs[1], "On")

    fig.savefig(f"./plots/{filename}")
    plt.show()

In [ ]:
%matplotlib inline
frequency_analysis_side_by_side(
    dfs_off.m2surr,
    dfs_on.m2surr,
    "SST M2 Surrogate",
    columns=["accelerationX", "accelerationY", "accelerationZ"],
    filename="sst_m2_surrogate_fft.png",
)

In [ ]:
%matplotlib inline
frequency_analysis_side_by_side(
    dfs_off.spider,
    dfs_on.spider,
    "SST spider spindle",
    ["accelerationX", "accelerationY", "accelerationZ"],
    filename="sst_spider_spindle_fft.png",
)

In [ ]:
%matplotlib inline
frequency_analysis_side_by_side(
    dfs_off.ter_pxmy,
    dfs_on.ter_pxmy,
    "SST top end ring +x -y",
    ["accelerationX", "accelerationY", "accelerationZ"],
    filename="sst_top_end_ring_plusX_minusY.png",
)

In [ ]:
%matplotlib inline
frequency_analysis_side_by_side(
    dfs_off.ter_mxmy,
    dfs_on.ter_mxmy,
    "SST top end ring -x -y",
    ["accelerationX", "accelerationY", "accelerationZ"],
    filename="sst_top_end_ring_minusX_minusY.png",
)

In [ ]:
%matplotlib inline
frequency_analysis_side_by_side(
    dfs_off.m2, dfs_on.m2, "M2 IMS", ["X", "Y", "Z"]
)

In [ ]:
dfs_on.m2